In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# input
df = pd.read_csv(snakemake.input.recall_per_sample, sep="\t")
df = df[["tool", "sample", "recalls_wrt_truth_probes"]]
tools_to_keep = snakemake.wildcards.tools_to_keep
tools_to_keep = tools_to_keep.split("_")
df

,tool,sample,recalls_wrt_truth_probes
0,snippy_NC_004431.1,Escherichia_coli_MSB1_6C,0.821731
1,snippy_NC_010498.1,Escherichia_coli_MINF_7C,0.889135
2,pandora_illumina_nodenovo_global_genotyping,Escherichia_coli_MSB1_7C,0.887915
3,snippy_NZ_CP008697.1,Escherichia_coli_MSB2_1A,0.885606
4,samtools_NZ_CP013483.1,Escherichia_coli_MSB1_4I,0.804207
...,...,...,...
1035,snippy_CP018206.1,Escherichia_coli_MSB1_3B,0.927632
1036,pandora_illumina_nodenovo_global_genotyping,Escherichia_coli_MINF_1D,0.895750
1037,samtools_NZ_CP008697.1,Escherichia_coli_MSB1_1A,0.912384
1038,snippy_CP010116.1,CFT073,0.813402


In [3]:
# filtering for tools to keep
def tool_is_inside_tools_to_keep(tool):
    for tool_to_keep in tools_to_keep:
        if tool.startswith(tool_to_keep):
            return True
    return False

df = df[[tool_is_inside_tools_to_keep(tool) for tool in df.tool]]
df

,tool,sample,recalls_wrt_truth_probes
0,snippy_NC_004431.1,Escherichia_coli_MSB1_6C,0.821731
1,snippy_NC_010498.1,Escherichia_coli_MINF_7C,0.889135
2,pandora_illumina_nodenovo_global_genotyping,Escherichia_coli_MSB1_7C,0.887915
3,snippy_NZ_CP008697.1,Escherichia_coli_MSB2_1A,0.885606
11,snippy_CP010121.1,CFT073,0.844107
...,...,...,...
1031,snippy_NZ_CP008697.1,Escherichia_coli_MSB1_3B,0.894887
1035,snippy_CP018206.1,Escherichia_coli_MSB1_3B,0.927632
1036,pandora_illumina_nodenovo_global_genotyping,Escherichia_coli_MINF_1D,0.895750
1038,snippy_CP010116.1,CFT073,0.813402


In [4]:
# create ref and tool columns


# get ref out of tool column
def get_ref(tool):
    if tool.startswith("pandora"):
        return "PRG"
    else:
        return tool[tool.index("_")+1:]

    
def get_tool(tool):
    if "pandora" in tool:
        if "nodenovo" in tool or "no_denovo" in tool:
            if "illumina" in tool:
                return "Pandora illumina no denovo"
            elif "nanopore" in tool:
                return "Pandora nanopore no denovo"
            else:
                raise RuntimeError(f"Unknown tool: {tool}")
        elif "withdenovo" in tool or "with_denovo" in tool:
            if "illumina" in tool:
                return "Pandora illumina with denovo"
            elif "nanopore" in tool:
                return "Pandora nanopore with denovo"
            else:
                raise RuntimeError(f"Unknown tool: {tool}")
    elif "snippy" in tool:
        return "Snippy"
    elif "samtools" in tool:
        return "Samtools"
    elif "medaka" in tool:
        return "Medaka"
    elif "nanopolish" in tool:
        return "Nanopolish"
    else:
        raise RuntimeError(f"Unknown tool: {tool}")


df["ref"] = df["tool"].apply(get_ref)
df["tool"] = df["tool"].apply(get_tool)
df

/home/leandro/git/pandora1_paper/venv/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/leandro/git/pandora1_paper/venv/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/leandro/git/pandora1_paper/venv/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,tool,sample,recalls_wrt_truth_probes,ref
0,Snippy,Escherichia_coli_MSB1_6C,0.821731,NC_004431.1
1,Snippy,Escherichia_coli_MINF_7C,0.889135,NC_010498.1
2,Pandora illumina no denovo,Escherichia_coli_MSB1_7C,0.887915,PRG
3,Snippy,Escherichia_coli_MSB2_1A,0.885606,NZ_CP008697.1
11,Snippy,CFT073,0.844107,CP010121.1
...,...,...,...,...
1031,Snippy,Escherichia_coli_MSB1_3B,0.894887,NZ_CP008697.1
1035,Snippy,Escherichia_coli_MSB1_3B,0.927632,CP018206.1
1036,Pandora illumina no denovo,Escherichia_coli_MINF_1D,0.895750,PRG
1038,Snippy,CFT073,0.813402,CP010116.1


In [5]:
# add canonical names to refs

def get_canonical_ref_names(ref):
    if ref=="PRG":
        return "PRG"
    
    ref_to_canonical = {
        "CP010116.1": "C1",
        "CP010121.1": "C4",
        "CP010170.1": "H6",
        "CP010171.1": "H7",
        "CP010226.1": "S1",
        "CP010230.1": "S21",
        "CP018206.1": "MRSN346647",
        "CU928163.2": "UMN026",
        "NC_004431.1": "CFT073",
        "NC_007779.1": "W3110",
        "NC_010498.1": "SMS-3-5",
        "NC_011742.1": "S88",
        "NC_011993.1": "LF82",
        "NC_017646.1": "CE10",
        "NC_022648.1": "JJ1886",
        "NZ_CP008697.1": "ST648",
        "NZ_CP009859.1": "ECONIH1",
        "NZ_CP011134.1": "VR50",
        "NZ_CP013483.1": "Y5",
        "NZ_CP015228.1": "09-00049",
        "NZ_CP016007.1": "NGF1",
        "NZ_CP018109.1": "MRSN346595",
        "NZ_HG941718.1": "EC958",
        "NZ_LM995446.1": "EcRV308Chr",
        "NZ_LT632320.1": "NCTC_13441"
    }
    return f"{ref_to_canonical[ref]} ({ref})"

df["ref"] = df["ref"].apply(get_canonical_ref_names)
df

/home/leandro/git/pandora1_paper/venv/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,tool,sample,recalls_wrt_truth_probes,ref
0,Snippy,Escherichia_coli_MSB1_6C,0.821731,CFT073 (NC_004431.1)
1,Snippy,Escherichia_coli_MINF_7C,0.889135,SMS-3-5 (NC_010498.1)
2,Pandora illumina no denovo,Escherichia_coli_MSB1_7C,0.887915,PRG
3,Snippy,Escherichia_coli_MSB2_1A,0.885606,ST648 (NZ_CP008697.1)
11,Snippy,CFT073,0.844107,C4 (CP010121.1)
...,...,...,...,...
1031,Snippy,Escherichia_coli_MSB1_3B,0.894887,ST648 (NZ_CP008697.1)
1035,Snippy,Escherichia_coli_MSB1_3B,0.927632,MRSN346647 (CP018206.1)
1036,Pandora illumina no denovo,Escherichia_coli_MINF_1D,0.895750,PRG
1038,Snippy,CFT073,0.813402,C1 (CP010116.1)


In [6]:
# add tool_and_ref column
def get_tool_and_ref(df):
    return f"{df['tool']} / {df['ref']}"
    

df["tool_and_ref"] = df.apply(get_tool_and_ref, axis=1)
df

/home/leandro/git/pandora1_paper/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,tool,sample,recalls_wrt_truth_probes,ref,tool_and_ref
0,Snippy,Escherichia_coli_MSB1_6C,0.821731,CFT073 (NC_004431.1),Snippy / CFT073 (NC_004431.1)
1,Snippy,Escherichia_coli_MINF_7C,0.889135,SMS-3-5 (NC_010498.1),Snippy / SMS-3-5 (NC_010498.1)
2,Pandora illumina no denovo,Escherichia_coli_MSB1_7C,0.887915,PRG,Pandora illumina no denovo / PRG
3,Snippy,Escherichia_coli_MSB2_1A,0.885606,ST648 (NZ_CP008697.1),Snippy / ST648 (NZ_CP008697.1)
11,Snippy,CFT073,0.844107,C4 (CP010121.1),Snippy / C4 (CP010121.1)
...,...,...,...,...,...
1031,Snippy,Escherichia_coli_MSB1_3B,0.894887,ST648 (NZ_CP008697.1),Snippy / ST648 (NZ_CP008697.1)
1035,Snippy,Escherichia_coli_MSB1_3B,0.927632,MRSN346647 (CP018206.1),Snippy / MRSN346647 (CP018206.1)
1036,Pandora illumina no denovo,Escherichia_coli_MINF_1D,0.895750,PRG,Pandora illumina no denovo / PRG
1038,Snippy,CFT073,0.813402,C1 (CP010116.1),Snippy / C1 (CP010116.1)


In [7]:
# save csv
df.to_csv(snakemake.output.csv_data, index=False)